In [1]:
import collections
import random
import string
import itertools

In [2]:
Link = collections.namedtuple('Link', 'height left right')

In [3]:
def link_ends(link):
    return set((link.left, link.right))

In [4]:
def can_add(link, links):
    ends = link_ends(link)
    same_height_links = [l for l in links if l.height == link.height]
    return all(ends.isdisjoint(link_ends(l)) for l in same_height_links)

In [5]:
def make_net(num_links, lines=10, height=50):
    links = set()
    while len(links) < num_links:
        a = random.randrange(lines)
        b = random.randrange(lines)
        if a != b:
            l = min(a, b)
            r = max(a, b)
            h = random.randrange(height)
            link = Link(h, l, r)
            if can_add(link, links):
                links.add(link)
    return links

In [6]:
net = make_net(20)
net

{Link(height=2, left=0, right=3),
 Link(height=2, left=1, right=7),
 Link(height=3, left=1, right=8),
 Link(height=9, left=6, right=9),
 Link(height=15, left=0, right=4),
 Link(height=20, left=0, right=8),
 Link(height=23, left=3, right=4),
 Link(height=24, left=3, right=6),
 Link(height=28, left=1, right=8),
 Link(height=28, left=5, right=6),
 Link(height=32, left=1, right=7),
 Link(height=34, left=1, right=9),
 Link(height=36, left=0, right=4),
 Link(height=38, left=5, right=6),
 Link(height=40, left=0, right=2),
 Link(height=41, left=3, right=7),
 Link(height=43, left=2, right=4),
 Link(height=43, left=6, right=8),
 Link(height=44, left=0, right=6),
 Link(height=46, left=7, right=8)}

In [7]:
def follow(initial_line, links):
    line = initial_line
    heights = sorted(set(l.height for l in links))
    for h in heights:
        for l in [l for l in links if l.height == h]:
            if line in link_ends(l):
                line = [e for e in link_ends(l) if e != line][0]
#                 print(l, line)
    return line

In [8]:
follow(4, net)

3

In [9]:
def pack(net):
    packed_links = []
    line_heights = collections.defaultdict(lambda: -1)
    for link in sorted(net):
        link_height = max(line_heights[link.left], line_heights[link.right]) + 1
        line_heights[link.left] = link_height
        line_heights[link.right] = link_height
        packed_links += [Link(link_height, link.left, link.right)]
    return sorted(packed_links)

In [10]:
def follow_many_slow(in_sequence, links):
    out_sequence = [(follow(i, links), term) 
                    for i, term in enumerate(in_sequence)]
    return [term for i, term in sorted(out_sequence)]

In [11]:
def follow_many(in_sequence, net):
    height_groups = [list(g) for _, g in itertools.groupby(pack(net), lambda l: l.height)]
    seq = list(in_sequence)
    for links in height_groups:
        for link in links:
#             l = seq[link.left]
#             r = seq[link.right]
#             seq[link.right] = l
#             seq[link.left] = r
            seq[link.right], seq[link.left] = seq[link.left], seq[link.right]
    return seq

In [12]:
%%timeit
follow_many('abcdefghij', net)

10000 loops, best of 3: 45.7 µs per loop


In [13]:
# %%timeit
# follow_many_slow('abcdefghij', net)

In [104]:
def show_net(links, randomise=False, pair_sep=', '):
    if randomise:
        output = []
        heights = sorted(set(l.height for l in links))
        for h in heights:
            ls = [l for l in links if l.height == h]
            random.shuffle(ls)
            output += ['({}, {})'.format(l.left, l.right) for l in ls]
        return pair_sep.join(output)
    return pair_sep.join('({}, {})'.format(l.left, l.right) for l in sorted(links))

In [105]:
show_net(net)

'(0, 3), (1, 7), (1, 8), (6, 9), (0, 4), (0, 8), (3, 4), (3, 6), (1, 8), (5, 6), (1, 7), (1, 9), (0, 4), (5, 6), (0, 2), (3, 7), (2, 4), (6, 8), (0, 6), (7, 8)'

In [106]:
show_net(net, randomise=True)

'(1, 7), (0, 3), (1, 8), (6, 9), (0, 4), (0, 8), (3, 4), (3, 6), (5, 6), (1, 8), (1, 7), (1, 9), (0, 4), (5, 6), (0, 2), (3, 7), (6, 8), (2, 4), (0, 6), (7, 8)'

In [110]:
show_net(net, pair_sep=' : ')

'(0, 3) : (1, 7) : (1, 8) : (6, 9) : (0, 4) : (0, 8) : (3, 4) : (3, 6) : (1, 8) : (5, 6) : (1, 7) : (1, 9) : (0, 4) : (5, 6) : (0, 2) : (3, 7) : (2, 4) : (6, 8) : (0, 6) : (7, 8)'

In [111]:
show_net(net, pair_sep='\n')

'(0, 3)\n(1, 7)\n(1, 8)\n(6, 9)\n(0, 4)\n(0, 8)\n(3, 4)\n(3, 6)\n(1, 8)\n(5, 6)\n(1, 7)\n(1, 9)\n(0, 4)\n(5, 6)\n(0, 2)\n(3, 7)\n(2, 4)\n(6, 8)\n(0, 6)\n(7, 8)'

In [112]:
print(show_net(net, pair_sep='\n'))

(0, 3)
(1, 7)
(1, 8)
(6, 9)
(0, 4)
(0, 8)
(3, 4)
(3, 6)
(1, 8)
(5, 6)
(1, 7)
(1, 9)
(0, 4)
(5, 6)
(0, 2)
(3, 7)
(2, 4)
(6, 8)
(0, 6)
(7, 8)


In [108]:
show_net(net)

'(0, 3), (1, 7), (1, 8), (6, 9), (0, 4), (0, 8), (3, 4), (3, 6), (1, 8), (5, 6), (1, 7), (1, 9), (0, 4), (5, 6), (0, 2), (3, 7), (2, 4), (6, 8), (0, 6), (7, 8)'

In [17]:
ls = [l for l in net if l.height == 1]
ls

[]

In [18]:
random.shuffle(ls)

In [19]:
def read_net(net_string):
    return [Link(h, l, r) for h, (l, r) in enumerate(extract_pairs(net_string))]

In [20]:
def extract_pairs(net_string):
    return [[int(pi) for pi in p.split(', ')] for p in net_string[1:-1].split('), (')]

In [21]:
read_net(show_net(net))

[Link(height=0, left=0, right=3),
 Link(height=1, left=1, right=7),
 Link(height=2, left=1, right=8),
 Link(height=3, left=6, right=9),
 Link(height=4, left=0, right=4),
 Link(height=5, left=0, right=8),
 Link(height=6, left=3, right=4),
 Link(height=7, left=3, right=6),
 Link(height=8, left=1, right=8),
 Link(height=9, left=5, right=6),
 Link(height=10, left=1, right=7),
 Link(height=11, left=1, right=9),
 Link(height=12, left=0, right=4),
 Link(height=13, left=5, right=6),
 Link(height=14, left=0, right=2),
 Link(height=15, left=3, right=7),
 Link(height=16, left=2, right=4),
 Link(height=17, left=6, right=8),
 Link(height=18, left=0, right=6),
 Link(height=19, left=7, right=8)]

In [22]:
pack(net)

[Link(height=0, left=0, right=3),
 Link(height=0, left=1, right=7),
 Link(height=0, left=6, right=9),
 Link(height=1, left=0, right=4),
 Link(height=1, left=1, right=8),
 Link(height=2, left=0, right=8),
 Link(height=2, left=3, right=4),
 Link(height=3, left=0, right=4),
 Link(height=3, left=1, right=8),
 Link(height=3, left=3, right=6),
 Link(height=4, left=0, right=2),
 Link(height=4, left=1, right=7),
 Link(height=4, left=5, right=6),
 Link(height=5, left=1, right=9),
 Link(height=5, left=2, right=4),
 Link(height=5, left=3, right=7),
 Link(height=5, left=5, right=6),
 Link(height=6, left=6, right=8),
 Link(height=7, left=0, right=6),
 Link(height=7, left=7, right=8)]

In [23]:
pack(read_net(show_net(net)))

[Link(height=0, left=0, right=3),
 Link(height=0, left=1, right=7),
 Link(height=0, left=6, right=9),
 Link(height=1, left=0, right=4),
 Link(height=1, left=1, right=8),
 Link(height=2, left=0, right=8),
 Link(height=2, left=3, right=4),
 Link(height=3, left=0, right=4),
 Link(height=3, left=1, right=8),
 Link(height=3, left=3, right=6),
 Link(height=4, left=0, right=2),
 Link(height=4, left=1, right=7),
 Link(height=4, left=5, right=6),
 Link(height=5, left=1, right=9),
 Link(height=5, left=2, right=4),
 Link(height=5, left=3, right=7),
 Link(height=5, left=5, right=6),
 Link(height=6, left=6, right=8),
 Link(height=7, left=0, right=6),
 Link(height=7, left=7, right=8)]

In [24]:
pnet = pack(net)
rrnet = read_net(show_net(net, randomise=True))
rnet = read_net(show_net(net))
rnet == rrnet, pack(rrnet) == pnet

(False, True)

In [25]:
lnet = make_net(10207, 26, 100000)
plnet = pack(lnet)
assert follow_many(string.ascii_lowercase, lnet) == follow_many(string.ascii_lowercase, plnet)
# for i in range(204):
#     assert follow(i, lnet) == follow(i, plnet)

In [26]:
rlnet = read_net(show_net(lnet))
prlnet = pack(rlnet)

In [27]:
max(link.height for link in plnet)

2224

In [28]:
max(link.height for link in lnet)

99998

In [29]:
max(link.height for link in rlnet)

10206

In [30]:
max(link.height for link in prlnet)

2224

In [31]:
assert follow_many(string.ascii_lowercase, lnet) == follow_many(string.ascii_lowercase, prlnet)

In [32]:
%%timeit
follow_many(string.ascii_lowercase, lnet)

10 loops, best of 3: 23.4 ms per loop


In [33]:
# %%timeit
# follow_many_slow(string.ascii_lowercase, lnet)

In [34]:
def eliminable_pairs_slow(net):
    eps = []
    for l in net:
        o = Link(l.height + 1, l.left, l.right)
        if o in net:
            eps += [(l, o)]
    return eps    

In [35]:
def eliminable_pairs(net):
    height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}
    eps = []
    for h in range(1, max(height_groups.keys())):
        for l in height_groups[h]:
            o = Link(l.height - 1, l.left, l.right)
            if o in height_groups[h-1]:
                eps += [(l, o)]
    return eps

In [36]:
%%timeit
eliminable_pairs(plnet)

10 loops, best of 3: 23.5 ms per loop


In [37]:
def eliminable_pair(net):
    for l in net:
        o = Link(l.height + 1, l.left, l.right)
        if o in net:
            return l, o
    return None

In [38]:
def eliminable_pair_hg(height_groups):
    for h in range(1, max(height_groups.keys())):
        for l in height_groups[h]:
            o = Link(l.height - 1, l.left, l.right)
            if o in height_groups[h-1]:
                return l, o
    return None

In [39]:
def eliminate_pairs_slow(net):
    eliminable_links = eliminable_pair(net)
    while eliminable_links:
        net = pack(l for l in net if l not in eliminable_links)
        eliminable_links = eliminable_pair(net)
    return net

In [40]:
def eliminate_pairs(net):
    height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}
    eliminable_links = eliminable_pair_hg(height_groups)
    while eliminable_links:
        net = pack(l for l in net if l not in eliminable_links)
        height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}
        eliminable_links = eliminable_pair_hg(height_groups)
    return net

In [41]:
print(len(plnet))
elnet = eliminate_pairs(plnet)
len(plnet), len(elnet)

10207


(10207, 9839)

In [42]:
eliminable_pairs(elnet)

[]

In [43]:
assert follow_many(string.ascii_lowercase,  lnet) == follow_many(string.ascii_lowercase, elnet)
assert follow_many(string.ascii_lowercase, plnet) == follow_many(string.ascii_lowercase, elnet)

In [44]:
%%timeit
elnet = eliminate_pairs(plnet)

1 loop, best of 3: 5.77 s per loop


In [45]:
# for i in range(26):
#     assert follow(i, plnet) == follow(i, elnet)
#     assert follow(i,  lnet) == follow(i, elnet)

In [46]:
# follow(0, plnet), follow(0, elnet), follow(0, lnet)

In [47]:
def triple_slow(net):
    x = None
    y = None
    ts = []
    for a in net:
        bs = [l for l in net if l.height == a.height + 1 
              if l.left == a.right or l.right == a.left]
        for b in bs:
            c = Link(a.height + 2, a.left, a.right)
            if c in net:
                ts += [(a, b, c)]
    return ts

In [48]:
def triple_pair_slow(net):
    ts = []
    for a in net:
        a_ends = link_ends(a)
        bs = [l for l in net if l.height == a.height + 1 
              if link_ends(l) & a_ends]
        if len(bs) == 1:
            b = bs[0]
            lines = set((a.left, a.right, b.left, b.right))
            cs = [l for l in net 
                  if l.height == a.height + 2
                  if link_ends(l) & lines]
            if len(cs) == 1:
                c = Link(a.height + 2, a.left, a.right)
                if c in cs:
                    ds = [l for l in net 
                          if l.height == a.height + 3
                          if link_ends(l) & lines]
                    d = Link(a.height + 3, b.left, b.right)
                    if d in ds:
                        ts += [(a, b, c, d)]
    return ts

In [49]:
def find_height_groups(net):
    return {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}

In [50]:
def triple_pair_hg(height_groups, debug=False):
    ts = []
    for h in range(3, max(height_groups.keys())):
        for d in height_groups[h]:
            if debug: print('d:', d)
            ch = h - 1
            cs = [l for l in height_groups[ch] if link_ends(l) & link_ends(d)]
            if debug: print('cs:', cs)
            while ch > 2 and not cs:
                ch -= 1
                cs = [l for l in height_groups[ch] if link_ends(l) & link_ends(d)]
                if debug: print('cs:', cs)
            if len(cs) == 1:
                c = cs[0]
                lines = set((d.left, d.right, c.left, c.right))
                if debug: print('c:', '; lines:', lines)
                bs = [l for l in height_groups[ch-1] if link_ends(l) & lines]
                b = Link(ch - 1, d.left, d.right)
                if debug: print('b:', b, '; bs:', bs)
                if len(bs) == 1 and b in bs:
                    ah = b.height - 1
                    als = [l for l in height_groups[ah] if link_ends(l) & link_ends(c)]
                    if debug: print('ah:', ah, '; als:', als)
                    while ah > 0 and not als:
                        ah -= 1
                        als = [l for l in height_groups[ah] if link_ends(l) & link_ends(c)]
                        if debug: print('ah:', ah, '; als:', als)
                    a = Link(ah, c.left, c.right)
                    if debug: print('a:', a)
                    if a in als:
                        if debug: print('adding:', a, b, c, d)
                        ts += [(a, b, c, d)]
    return ts

In [51]:
def eliminate_a_triple_pair_slow(net, debug=False):
    tps = triple_pair_slow(net)
    if debug: print('eatp', tps)

    if tps:
        a, b, c, d = tps[0]
#         x = Link(a.height, b.left, b.right)
#         y = Link(b.height, a.left, a.right)
        x = Link(b.height - 0.5, b.left, b.right)
        y = Link(b.height, a.left, a.right)
        if debug: print('removing', a, b, c, d, '; adding', x, y)
        return pack([l for l in net if l not in [a, b, c, d]] + [x, y])
    return None

In [52]:
def eliminate_a_triple_pair(net, debug=False):
    height_groups = find_height_groups(net)

    tps = triple_pair_hg(height_groups)
    if debug: print('eatp', tps)
    if tps:
        a, b, c, d = tps[0]
        x = Link(b.height - 0.5, b.left, b.right)
        y = Link(b.height, a.left, a.right)
        if debug: print('removing', a, b, c, d, '; adding', x, y)
        return pack([l for l in net if l not in [a, b, c, d]] + [x, y])
    return None

In [53]:
height_groups = find_height_groups(elnet)
triple_pair_hg(height_groups)

[(Link(height=831, left=8, right=16),
  Link(height=832, left=8, right=25),
  Link(height=833, left=8, right=16),
  Link(height=834, left=8, right=25)),
 (Link(height=1657, left=1, right=13),
  Link(height=1658, left=1, right=12),
  Link(height=1659, left=1, right=13),
  Link(height=1660, left=1, right=12))]

In [54]:
%%timeit
height_groups = find_height_groups(elnet)
triple_pair_hg(height_groups)

10 loops, best of 3: 99.4 ms per loop


In [55]:
def eliminate_triple_pairs_slow(net):
    print(len(net))
    new_net = eliminate_a_triple_pair_slow(net)
    while new_net:
        print(len(net))
        net = new_net
        new_net = eliminate_a_triple_pair_slow(net)
    return net

In [56]:
def eliminate_triple_pairs(net):
    print(len(net))
    new_net = eliminate_a_triple_pair(net)
    while new_net:
        print(len(net))
        net = new_net
        new_net = eliminate_a_triple_pair(net)
    return net

In [57]:
etlnet = eliminate_a_triple_pair(elnet)

In [58]:
assert follow_many(string.ascii_lowercase, etlnet) == follow_many(string.ascii_lowercase, elnet)

In [59]:
setlnet = eliminate_triple_pairs(elnet)

9839
9839
9837


In [60]:
len(setlnet)

9835

In [61]:
assert follow_many(string.ascii_lowercase, etlnet) == follow_many(string.ascii_lowercase, elnet)

In [62]:
''.join(follow_many(string.ascii_lowercase, etlnet))

'buxphgtzqykfawvomcjresnldi'

In [63]:
''.join(follow_many(string.ascii_lowercase, setlnet))

'buxphgtzqykfawvomcjresnldi'

In [64]:
''.join(follow_many(string.ascii_lowercase, elnet))

'buxphgtzqykfawvomcjresnldi'

In [65]:
''.join(follow_many(string.ascii_lowercase, lnet))

'buxphgtzqykfawvomcjresnldi'

In [66]:
eliminable_pairs(etlnet)

[]

In [67]:
len(lnet), len(etlnet)

(10207, 9837)

In [68]:
def simplify(net0):
    netp = eliminate_pairs(net0)
    new_net = eliminate_a_triple_pair(netp)
    while new_net:
#         print('sipl', len(net0), len(netp), len(new_net))
        netp = eliminate_pairs(new_net)
        new_net = eliminate_a_triple_pair(netp)
    return netp

In [69]:
simple_lnet = simplify(plnet)

In [70]:
''.join(follow_many(string.ascii_lowercase, simple_lnet)) == ''.join(follow_many(string.ascii_lowercase, lnet))

True

In [71]:
''.join(follow_many(string.ascii_lowercase, simple_lnet))

'buxphgtzqykfawvomcjresnldi'

In [72]:
''.join(follow_many(string.ascii_lowercase, lnet))

'buxphgtzqykfawvomcjresnldi'

In [73]:
len(simple_lnet)

9835

In [89]:
def add_triple_pair(net, max_lines=None, trace=False):
    if not max_lines:
        max_lines = max(l.right for l in net)
    a, b, c = 0, 0, 0
    while len(set((a, b, c))) != 3:
        a = random.randrange(max_lines)
        b = random.randrange(max_lines)
        c = random.randrange(max_lines)
    tp = [(min(a, b), max(a, b)), (min(b, c), max(b, c))] * 2
    
    pairs = [(l.left, l.right) for l in sorted(net)]
    i = random.randrange(len(pairs))
    if trace: print(i, tp)
    new_pairs = pairs[:i] + tp + pairs[i:]
    return pack([Link(h, l, r) for h, (l, r) in enumerate(new_pairs)])   

In [90]:
def add_pair(net, max_lines=None, trace=False):
    if not max_lines:
        max_lines = max(l.right for l in net)

    a, b = 0, 0
    while a == b:
        a = random.randrange(max_lines)
        b = random.randrange(max_lines)
    p = [(min(a, b), max(a, b))] * 2
    
    pairs = [(l.left, l.right) for l in sorted(net)]
    i = random.randrange(len(pairs))
    
    if trace: print(i, p)
    new_pairs = pairs[:i] + p + pairs[i:]
    return pack([Link(h, l, r) for h, (l, r) in enumerate(new_pairs)])   

In [76]:
height_groups = {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}
tps = triple_pair_hg(height_groups)
tps

[]

In [91]:
lnettp = simple_lnet
for _ in range(10):
    lnettp = add_pair(lnettp)
height_groups = {h: list(g) for h, g in itertools.groupby(pack(lnettp), lambda l: l.height)}
tps = triple_pair_hg(height_groups)
tps

[]

In [92]:
lnettp = simple_lnet
for _ in range(10):
    lnettp = add_triple_pair(lnettp)
height_groups = {h: list(g) for h, g in itertools.groupby(pack(lnettp), lambda l: l.height)}
tps = triple_pair_hg(height_groups)
tps

[(Link(height=195, left=5, right=10),
  Link(height=197, left=1, right=5),
  Link(height=198, left=5, right=10),
  Link(height=199, left=1, right=5)),
 (Link(height=359, left=6, right=16),
  Link(height=360, left=16, right=20),
  Link(height=361, left=6, right=16),
  Link(height=362, left=16, right=20)),
 (Link(height=409, left=0, right=9),
  Link(height=410, left=9, right=16),
  Link(height=411, left=0, right=9),
  Link(height=412, left=9, right=16)),
 (Link(height=606, left=0, right=7),
  Link(height=607, left=0, right=5),
  Link(height=608, left=0, right=7),
  Link(height=609, left=0, right=5)),
 (Link(height=967, left=7, right=19),
  Link(height=968, left=19, right=23),
  Link(height=969, left=7, right=19),
  Link(height=970, left=19, right=23)),
 (Link(height=973, left=9, right=18),
  Link(height=975, left=9, right=15),
  Link(height=976, left=9, right=18),
  Link(height=977, left=9, right=15)),
 (Link(height=1193, left=1, right=11),
  Link(height=1194, left=1, right=19),
  Link(h

In [93]:
lnettp = simple_lnet
for _ in range(50):
    lnettp = add_pair(lnettp)
for _ in range(50):
    lnettp = add_triple_pair(lnettp)
height_groups = {h: list(g) for h, g in itertools.groupby(pack(lnettp), lambda l: l.height)}
tps = triple_pair_hg(height_groups)
tps

[(Link(height=8, left=1, right=5),
  Link(height=9, left=1, right=21),
  Link(height=10, left=1, right=5),
  Link(height=11, left=1, right=21)),
 (Link(height=40, left=16, right=23),
  Link(height=41, left=16, right=19),
  Link(height=42, left=16, right=23),
  Link(height=43, left=16, right=19)),
 (Link(height=62, left=0, right=10),
  Link(height=63, left=10, right=13),
  Link(height=64, left=0, right=10),
  Link(height=65, left=10, right=13)),
 (Link(height=137, left=23, right=24),
  Link(height=139, left=0, right=24),
  Link(height=140, left=23, right=24),
  Link(height=141, left=0, right=24)),
 (Link(height=138, left=10, right=21),
  Link(height=139, left=2, right=10),
  Link(height=140, left=10, right=21),
  Link(height=141, left=2, right=10)),
 (Link(height=139, left=2, right=10),
  Link(height=140, left=10, right=21),
  Link(height=141, left=2, right=10),
  Link(height=142, left=10, right=21)),
 (Link(height=156, left=6, right=11),
  Link(height=157, left=3, right=6),
  Link(heig

In [94]:
lnettp == pack(lnettp)

True

In [95]:
lnettps = simplify(lnettp)
len(lnettps)

9931

In [96]:
len(simple_lnet), len(lnettp), len(lnettps)

(9835, 10135, 9931)

In [97]:
''.join(follow_many(string.ascii_lowercase, lnet)) == ''.join(follow_many(string.ascii_lowercase, simple_lnet))

True

In [98]:
''.join(follow_many(string.ascii_lowercase, lnettps)) == ''.join(follow_many(string.ascii_lowercase, lnettp))

True

In [99]:
max(l.height for l in lnettp)

2286

In [100]:
def simplify_with_checks(net0):
    netp = eliminate_pairs(net0)
    if follow_many(string.ascii_lowercase, net0) != follow_many(string.ascii_lowercase, netp):
        print('pairs', eliminable_pairs(net0))
        return net0
    else:
        print('pairs ok')
    new_net = eliminate_a_triple_pair(netp)
    if new_net and follow_many(string.ascii_lowercase, new_net) != follow_many(string.ascii_lowercase, netp):
        hg = find_height_groups(netp)
        print('triple', triple_pair_hg(hg))
        return netp
    else:
        print('triple ok')
    while new_net:
#         print('sipl', len(net0), len(netp), len(new_net))
        netp = eliminate_pairs(new_net)
        if follow_many(string.ascii_lowercase, new_net) != follow_many(string.ascii_lowercase, netp):
            print('pairs', eliminable_pairs(new_net))
            return new_net
        else:
            print('pairs ok')
        new_net = eliminate_a_triple_pair(netp)
        if new_net and follow_many(string.ascii_lowercase, new_net) != follow_many(string.ascii_lowercase, netp):
            hg = find_height_groups(netp)
            print('triple', triple_pair_hg(hg))
            return netp
        else:
            print('triple ok')
    print('** done')
    return netp

In [101]:
lnettps = simplify_with_checks(lnettp)

pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
pairs ok
triple ok
** done


In [102]:
''.join(follow_many(string.ascii_lowercase, lnettps)) == ''.join(follow_many(string.ascii_lowercase, lnettp))

True

In [103]:
len(lnettp)

10135

In [113]:
open('04-lines.txt', 'w').write(show_net(lnettp, randomise=True, pair_sep='\n'))

83459

In [114]:
open('04-small.txt', 'w').write(show_net(make_net(20), randomise=True, pair_sep='\n'))

139